## Stanford Cars

* Data set: https://ai.stanford.edu/~jkrause/cars/car_dataset.html
* Related papers: http://cs231n.stanford.edu/reports/2015/pdfs/lediurfinal.pdf, http://noiselab.ucsd.edu/ECE228/Reports/Report17.pdf

### Similar projects

* 88% accuracy with resnet152 https://github.com/foamliu/Car-Recognition
* Kaggle solution with 90% accuracy: https://www.kaggle.com/meaninglesslives/cars-eb0-keras

### Notes for running on Databricks

* Requires cluster enabled for JupyterLab support
* Use Latest experimental GPU ML Runtime (6.6 snapshot)
* Install libraries
    * opencv-python==4.0.0.21
* Post issues to [this email thread](https://groups.google.com/a/databricks.com/d/msgid/ml-sme/CA%2BUeztiEsUTm2xEZnBZp2DOgiWocCkJ%3DLNo6q1-Fn3%2BXdN4prQ%40mail.gmail.com?utm_medium=email&utm_source=footer) [internal link]

## Imports

In [2]:
import scipy.io as sio
import numpy as np
from IPython.display import Image
import os
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import keras
from keras.callbacks import ModelCheckpoint
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.pooling import GlobalAveragePooling2D, AveragePooling2D
from keras import applications  # these are the applications built into keras
from keras_applications.resnet import ResNet152 # separate keras applications lib, seems more up to date
keras.backend.tensorflow_backend._get_available_gpus()

Command execution completed

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
%load_ext autoreload
%autoreload 2

Command execution completed

/databricks/python/lib/python3.7/site-packages/tensorflow_core/contrib/learn/python/learn/learn_io/generator_io.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Container


### Settings + globals

In [4]:
# Running on databricks with DBFS 
use_dbfs = True

# Set to True if you need to convert the original images into the squashed 227x227 images.
# If you already have the squashed 227x227 images in cars_train_227_227, no need to run this. 
do_image_preprocessing = False

Command execution completed

## Data preparation

### Copy dataset from DBFS -> local instance storage

In [5]:
if use_dbfs and not os.path.exists("/home/ubuntu/datasets/stanfordcars"):
    print("Copying dataset from DBFS -> local instance storage..")
    !cp -r dbfs/traun.leyden/datasets/stanfordcars /home/ubuntu/datasets/
    !ls /home/ubuntu/datasets/
    !mkdir dbfs/traun.leyden/models
    !mkdir /home/ubuntu/models
    print("Finished copying dataset from DBFS -> local instance storage")
    !ls -al /home/ubuntu/datasets/

Command execution completed

Copying dataset from DBFS -> local instance storage..
README.txt     cars_test_227_227    cars_train_227_227
cars_meta.mat  cars_test_annos.mat  cars_train_annos.mat
mkdir: cannot create directory ‘dbfs/traun.leyden/models’: File exists
Finished copying dataset from DBFS -> local instance storage
total 896
drwxr-xr-x 4 root   root     4096 Mar 10 16:00 .
drwxr-xr-x 1 ubuntu ubuntu   4096 Mar 10 16:00 ..
-rwxr-xr-x 1 root   root     6148 Mar 10 16:00 .DS_Store
-rwxr-xr-x 1 root   root      120 Mar 10 16:00 ._.DS_Store
-rwxr-xr-x 1 root   root      482 Mar 10 16:00 ._cars_test_annos.mat
-rwxr-xr-x 1 root   root      218 Mar 10 16:00 .floyddata
-rwxr-xr-x 1 root   root     1654 Mar 10 16:00 README.txt
-rwxr-xr-x 1 root   root     3177 Mar 10 16:00 cars_meta.mat
drwxr-xr-x 2 root   root   245760 Mar 10 15:53 cars_test_227_227
-rwxr-xr-x 1 root   root   185758 Mar 10 16:00 cars_test_annos.mat
drwxr-xr-x 2 root   root   253952 Mar 10 16:00 cars_train_227_227
-rwxr-xr-x 1 root   root   187916

### Data helper functions

In [6]:
def ensure_exists(path):
    if not os.path.exists(path):
        raise Exception("Could not find path: {}".format(path))
        
def get_class(car):
    """
    Helper function to convert a raw "car" stored in matlab format into
    a dictionary w/ named fields
    """
    filename = car[5][0].item()
    class_id = car[4][0][0].item()
    bbox = {
        "x1": car[0][0][0].item(),
        "y1": car[1][0][0].item(),
        "x2": car[2][0][0].item(),
        "y2": car[3][0][0].item()
    }
    class_ = classes[car[4][0][0]]
    return {
        "filename":filename, 
        "class_id": class_id,
        "class": class_, 
        "bbox": bbox
    }

Command execution completed

### Load annotations from matlab files

In [7]:

if use_dbfs:
    datadir = "/home/ubuntu/datasets"
else:
    datadir = "datasets/StanfordCars"

cars_train_227_227 = os.path.join(datadir, "cars_train_227_227")
cars_test_227_227 = os.path.join(datadir, "cars_test_227_227")
ensure_exists(cars_train_227_227)
ensure_exists(cars_test_227_227)

# Annotations (matlab)
cars_meta = sio.loadmat(datadir + "/cars_meta.mat")
cars_train = sio.loadmat(datadir + "/cars_train_annos.mat")
cars_test = sio.loadmat(datadir + "/cars_test_annos.mat")

# Car classes
classes = [None] # MatLab is 1-based, python 0-based
classes += [c[0].item() for c in cars_meta["class_names"][0]] 

# Training and testing annotations (matlab)
training_annotations = cars_train['annotations'][0]
test_annotations = cars_test['annotations'][0]

Command execution completed

## Image preprocessing pipeline


### Crop with boundary

From the Lieu/Wang paper:

> To preserve some context surrounding the cars, we expanded each bounding box by 16 pixels on each side before cropping

### Resize to 227x227 square aspect ratio

From the Lieu/Wang paper:


> we resized each cropped image to a square aspect ratio and a resolution of 227x227
as required by the models. After discussions with Krause, we decided to squash images without preserving their original aspect ratios instead of scaling and cropping the image

### Image processing helpers

In [8]:
def crop_expand_bounding_box(car_class, source_dir):
    
    """
    Given a car class:
    
    {'filename': '00003.jpg',
     'class_id': 145,
     'class': 'Jeep Patriot SUV 2012',
     'bbox': {'x1': 51, 'y1': 105, 'x2': 968, 'y2': 659}}
     
    And an source and output directory, do the following:
    
    1. Calculate the expanded bounding box (should not go outside image border)
    2. Crop the image with the expanding box
    3. Return cropped image
    """
    source_filename = "{}/{}".format(source_dir, car_class['filename'])
    
    if not os.path.exists(source_filename):
        raise Exception("Could not find source image file: {}".format(source_filename))
        
    source_img = cv2.imread(source_filename)
    height, width, channels = source_img.shape
    bbox_orig = car_class['bbox']
    bbox = expand_bounding_box(bbox_orig, (width, height), 16)
    cropped_img = source_img[bbox['y1']:bbox['y2'], bbox['x1']:bbox['x2']]
    return cropped_img

def expand_bounding_box(bounding_box, img_size, expand_pixels):
    
    """
    Given a bounding box:
    
    {'x1': 51, 'y1': 105, 'x2': 968, 'y2': 659}
    
    an image size tuple (width, height) and a number of pixels to expand (expand_pixels param)
    
    Return a larger bounding box that still fits within the image bounds.
    
    """
    width, height = img_size
    new_x1 = max(bounding_box['x1'] - expand_pixels,0)  # don't let the new_x1 go off left edge of image
    new_x2 = min(bounding_box['x2'] + expand_pixels, width)  # don't let new_x2 go off right edge of image
    new_y1 = max(bounding_box['y1'] - expand_pixels, 0)  # don't go off top edge of image
    new_y2 = min(bounding_box['y2'] + expand_pixels, height)  # don't go off bottom edge of image
    
    return {
        'x1': new_x1,
        'y1': new_y1,
        'x2': new_x2,
        'y2': new_y2,
    }


def process_cars(cars, source_dir, result_directory_path):
    """
    Loop over car_classes and write transformed image into result_directory_path
    """
    for car in cars:
        car_class = get_class(car)
        print("car_class: {}".format(car_class))
        cropped_img = crop_expand_bounding_box(car_class, source_dir)
        resized_img = cv2.resize(cropped_img, (227,227))
        target_file = os.path.join(result_directory_path, car_class['filename'])
        cv2.imwrite(target_file, resized_img)
        
def process_car():
    source_dir = os.path.join(datadir, "cars_test")
    cropped_img = crop_expand_bounding_box(car_class, source_dir)

    img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
        

Command execution completed

### Do image processing

In [9]:
if do_image_preprocessing:
    source_dir = os.path.join(datadir, "cars_train")
    process_cars(training_annotations, source_dir, cars_train_227_227)
    source_dir = os.path.join(datadir, "cars_test")
    process_cars(test_annotations, source_dir, cars_test_227_227)
    

Command execution completed

## Keras ImageDataGenerator

### Based on tutorials/docs

* [Vijayabhaskar J's Tutorial on Keras flow_from_dataframe](https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c)

In [10]:
def dataframes_from_annotations(cars):
    """
    Given the annotations in matlab/octave format, create dataframes
    """
    dataframe = pd.DataFrame(columns=['id', 'label'])
    
    for car in cars:
        # Example car_class: {'filename': '00001.jpg', 'class_id': 14, 'class': 'Audi TTS Coupe 2012', 'bbox': {..}}
        car_class = get_class(car)
        dataframe = dataframe.append(
            {"id": car_class['filename'], 
             "label": car_class['class'],
            }, 
            ignore_index=True,
        )
    
    return dataframe
    

Command execution completed

In [11]:
training_dataframes = dataframes_from_annotations(training_annotations)
training_dataframes

Command execution completed

,id,label
0,00001.jpg,Audi TTS Coupe 2012
1,00002.jpg,Acura TL Sedan 2012
2,00003.jpg,Dodge Dakota Club Cab 2007
3,00004.jpg,Hyundai Sonata Hybrid Sedan 2012
4,00005.jpg,Ford F-450 Super Duty Crew Cab 2012
5,00006.jpg,Geo Metro Convertible 1993
6,00007.jpg,Dodge Journey SUV 2012
7,00008.jpg,Dodge Charger Sedan 2012
8,00009.jpg,Mitsubishi Lancer Sedan 2012
9,00010.jpg,Chevrolet Traverse SUV 2012


In [12]:
test_dataframes = dataframes_from_annotations(test_annotations)
test_dataframes

Command execution completed

,id,label
0,00001.jpg,Suzuki Aerio Sedan 2007
1,00002.jpg,Ferrari 458 Italia Convertible 2012
2,00003.jpg,Jeep Patriot SUV 2012
3,00004.jpg,Toyota Camry Sedan 2012
4,00005.jpg,Tesla Model S Sedan 2012
5,00006.jpg,Chrysler Town and Country Minivan 2012
6,00007.jpg,GMC Terrain SUV 2012
7,00008.jpg,Mercedes-Benz S-Class Sedan 2012
8,00009.jpg,BMW X5 SUV 2007
9,00010.jpg,Chevrolet HHR SS 2010


### Training/validation ImageDataGenerator helper functions

In [13]:
batch_size = 16
num_classes = 196 # the number of different cars
img_width = 227
img_height = 227

datagen=ImageDataGenerator(
    rescale=1./255.,
    validation_split=0.25,
    rotation_range=20.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

def get_train_generator(shuffle=False):
    train_generator=datagen.flow_from_dataframe(
        dataframe=training_dataframes,
        directory=cars_train_227_227,
        x_col='id',
        y_col='label',
        subset="training",
        batch_size=batch_size,
        shuffle=shuffle,
        seed=42,
        class_mode="categorical",
        target_size=(img_width,img_height),
    )
    return train_generator

def get_validation_generator(shuffle=False):
    validation_generator=datagen.flow_from_dataframe(
        dataframe=training_dataframes,
        directory=cars_train_227_227,
        x_col='id',
        y_col='label',
        subset="validation",
        batch_size=batch_size,
        shuffle=shuffle,
        seed=42,
        class_mode="categorical",
        target_size=(img_width,img_height),
    )
    return validation_generator

def get_test_generator(shuffle=False,classes=None):
    """
    This must take the "classes" as a param, which is a list of all the class labels:
    
        ['Audi TTS Coupe 2012', 'Acura TL Sedan 2012']
    
    Where the order is very important, because it's used to generate the one-hot
    encoded labels.  If the one-hot encoded labels are misaligned across the
    DataFrameIterator (training, validation, and test) then you will get totally
    wonky and invalid results.  This is required since the test set DataFrameIterators 
    uses it's own ImageDataGenerator separate from the one used by the training and 
    validation generators.
    """
    test_datagen=ImageDataGenerator(rescale=1./255.)
    test_generator=test_datagen.flow_from_dataframe(
        dataframe=test_dataframes,
        directory=cars_test_227_227,
        x_col='id',
        y_col='label',
        classes=classes,
        batch_size=batch_size,
        shuffle=shuffle,
        seed=42,
        class_mode="categorical",
        target_size=(img_width,img_height),
    )
    return test_generator




Command execution completed

### Instantiate training/validation ImageDataGenerators


In [14]:
train_generator_non_shuffle = get_train_generator(shuffle=False)
train_generator = get_train_generator(shuffle=True)
validation_generator_non_shuffle = get_validation_generator(shuffle=False)
validation_generator = get_validation_generator(shuffle=True)

# Use the classes from any of the above DataFrameIterators for the
# the test set DataFrameIterator.
classes = list(train_generator_non_shuffle.class_indices.keys())
test_generator = get_test_generator(shuffle=False, classes=classes)

steps_per_epoch_training=train_generator_non_shuffle.n // train_generator_non_shuffle.batch_size
steps_per_epoch_validation=validation_generator_non_shuffle.n // validation_generator_non_shuffle.batch_size
steps_per_epoch_test=test_generator.n // test_generator.batch_size
print("steps_per_epoch_training: {}".format(steps_per_epoch_training))
print("steps_per_epoch_validation: {}".format(steps_per_epoch_validation))

Command execution completed

Found 6108 validated image filenames belonging to 196 classes.
Found 6108 validated image filenames belonging to 196 classes.
Found 2036 validated image filenames belonging to 196 classes.
Found 2036 validated image filenames belonging to 196 classes.
Found 8041 validated image filenames belonging to 196 classes.
steps_per_epoch_training: 381
steps_per_epoch_validation: 127


### Training helper functions

In [15]:
def generator_with_labels(model, generator):
    """
    Helper which is an alternative to using model.predict_generator() which 
    has the advantage of also capturing the labels.
    See https://stackoverflow.com/questions/44970445/how-to-return-true-labels-of-items-when-using-predict-generator
    """
    while True:
        x, y = generator.next()
        yield x, model.predict_on_batch(x), y
        
def training_last_cnn_layer_with_labels(model, image_data_generator, steps_per_epoch):
        
    image_data_generator_w_labels = generator_with_labels(
        model, 
        image_data_generator,
    )
    
    num_steps_taken = 0
    y_preds = []
    y_labels = []
    for x, y_pred, y_label in image_data_generator_w_labels:
        print("{}/{}".format(num_steps_taken, steps_per_epoch))
        y_preds.append(y_pred)
        y_labels.append(y_label)
        num_steps_taken += 1
        if num_steps_taken >= steps_per_epoch:
            break
            
    return y_preds, y_labels

Command execution completed

## Transfer learning on resnet-156

According to http://noiselab.ucsd.edu/ECE228/Reports/Report17.pdf, they were only able to get ~50% test set accuracy on VGG16.

The approach below takes a very similar approach as https://github.com/foamliu/Car-Recognition, with the differences being:

* Use the keras application predefined model, whereas github/foamliu defines a custom model from scratch
* This model uses a dropout layer, github/foamli does not
* github/foamli passes in a ReduceLROnPlateau callback, this model does not (yet)
* github/foamli uses image folders rather than labels in a separate file, this notebook uses Pandas dataframes that are exported from the Matlab files in the original dataset.  (which shouldn't make any significant difference in results)


### Instantiate training/validation ImageDataGenerators

Cannot re-use training generators from above, since they are already exhausted

In [16]:
train_generator_non_shuffle = get_train_generator(shuffle=False)
train_generator = get_train_generator(shuffle=True)
validation_generator_non_shuffle = get_validation_generator(shuffle=False)
validation_generator = get_validation_generator(shuffle=True)

# Use the classes from any of the above DataFrameIterators for the
# the test set DataFrameIterator.
classes = list(train_generator_non_shuffle.class_indices.keys())
test_generator = get_test_generator(shuffle=False, classes=classes)

steps_per_epoch_training=train_generator_non_shuffle.n // train_generator_non_shuffle.batch_size
steps_per_epoch_validation=validation_generator_non_shuffle.n // validation_generator_non_shuffle.batch_size
steps_per_epoch_test=test_generator.n // test_generator.batch_size
print("steps_per_epoch_training: {}".format(steps_per_epoch_training))
print("steps_per_epoch_validation: {}".format(steps_per_epoch_validation))

Command execution completed

Found 6108 validated image filenames belonging to 196 classes.
Found 6108 validated image filenames belonging to 196 classes.
Found 2036 validated image filenames belonging to 196 classes.
Found 2036 validated image filenames belonging to 196 classes.
Found 8041 validated image filenames belonging to 196 classes.
steps_per_epoch_training: 381
steps_per_epoch_validation: 127


### Define transfer learning model

In [17]:
# build the network
base_model_resnet152 = ResNet152(
    weights='imagenet', 
    input_shape=(img_width, img_height, 3), 
    include_top=False,
    backend=keras.backend,  # workaround keras issue: https://github.com/keras-team/keras-applications/issues/54#issuecomment-445097297
    layers=keras.layers, 
    models=keras.models, 
    utils=keras.utils,
)

Command execution completed







234700800/234698864 [==============================] - 26s 0us/step




### Fine tune resnet152

### Define model

In [18]:

# Base resnet model
# -----------------
x = base_model_resnet152.output

# Add a new "top layer"
# --------------------

# Without this step, the memory requirements are huge and blows up an 8GB Geforce 1070 GPU
# but it might be possible to reduce the pool size for larger GPUs
x = AveragePooling2D(pool_size=(7, 7), data_format='channels_last')(x)

# Flatten the input to prep it for softmax dense layer
x = Flatten()(x)       

# Add healthy amount of dropout to avoid overfitting
x = Dropout(0.60)(x)

# Softmax 
preds = Dense(num_classes, activation='softmax')(x)

# Combine base model and top layer
# --------------------------------
combined_model_resnet152 = keras.Model(
    inputs=base_model_resnet152.input, 
    outputs=preds
)

Command execution completed


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Train model

In [19]:

# SGD optimizer
sgd = keras.optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model
combined_model_resnet152.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

# Model checkpointing callback
trained_models_path = '/home/ubuntu/models/stanfordcars'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, monitor='val_acc', verbose=1, save_best_only=True)
callbacks = [model_checkpoint]

# Train the model to fit the training data and compare against validation set
combined_model_resnet152.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    callbacks=callbacks,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation
)

Command execution completed



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
381/381 [==============================] - 594s 2s/step - loss: 5.3254 - acc: 0.0089 - val_loss: 5.1171 - val_acc: 0.0266

Epoch 00001: val_acc improved from -inf to 0.02657, saving model to /home/ubuntu/models/stanfordcars.01-0.03.hdf5
Epoch 2/30
381/381 [==============================] - 552s 1s/step - loss: 4.8416 - acc: 0.0449 - val_loss: 4.1641 - val_acc: 0.1312

Epoch 00002: val_acc improved from 0.02657 to 0.13119, saving model to /home/ubuntu/models/stanfordcars.02-0.13.hdf5
Epoch 3/30
381/381 [==============================] - 553s 1s/step - loss: 3.6823 - acc: 0.1808 - val_loss: 3.0734 - val_acc: 0.2649

Epoch 00003: val_acc improved from 0.13119 to 0.26485, saving model to /home/ubuntu/models/stanfordcars.03-0.26.hdf5
Epoch 4/30
381/381 [==============================] - 553s 1s/step - loss: 2.4635 - acc: 0.3840 - val_loss: 2.1519 - val_acc: 0.4554

Epoch 00004: val_a

### Copy trained model to DBFS

In [20]:
if use_dbfs:
    !cp /home/ubuntu/models/* dbfs/traun.leyden/models/

Command execution completed

### Run model against test set with unseen data

In [21]:
test_loss, test_accuracy = combined_model_resnet152.evaluate_generator(
    generator = test_generator,
    steps = steps_per_epoch_test,
    verbose = 1,
)
print("test_loss: {}, test_accuracy: {}".format(test_loss, test_accuracy))

Command execution completed

502/502 [==============================] - 201s 400ms/step
test_loss: 0.6507449657762928, test_accuracy: 0.8580677290836654
